# DISCLAIMER: Use litellm as chatmodel within langchain to enable easy switching between models





####langchain gemin hello

In [ ]:
!pip install -U langchain-google-genai

from google.colab import userdata
import os
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(model="gemini-pro")
print(llm.invoke("Sing a ballad of LangChain.").content)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "What's in this image?",
        },  # You can optionally provide text parts
        {"type": "image_url", "image_url": "https://picsum.photos/seed/picsum/200/300"},
    ]
)
print(llm.invoke([message]).content)

#RAG

## Making a rag system (in langchain)
### 1) create the vectorstore
### 2) make llm
### 3) query

## making vector store

In [ ]:
%pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:
import os
import time

from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "hello"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
import os
from google.colab import userdata

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

## making llm

In [ ]:
%pip install -qU langchain-google-genai
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## querying

In [ ]:
query: str = "tell about shark"

results = vector_store.similarity_search(
    query,
    k=1,
    filter={"source": "tweet"},
)

In [ ]:
messages = [
    (
        "system",
        "Answer the query using the references mentioned only",
    ),
    ("human", f"Query: {query}. References: {results}"),
]
ai_msg = llm.invoke(messages)

In [ ]:
print(ai_msg.content)

# ADV RAG
## 1) make the vector store
## 2) make the retriever
## 3) make the llm
## 4) make the prompt
## 5) make the chain
## 6) query

## making vector store

In [ ]:
!pip install -qU "langchain-chroma>=0.1.2"

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
import os
from google.colab import userdata

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="first",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

## making retriever

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)  # select top result

## making llm

In [ ]:
%pip install -qU langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## making prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

## chaining

In [ ]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

## querying

In [ ]:
response = rag_chain.invoke("tell about shark")

print(response.content)

# ADV RAG USING DOC

In [ ]:
!pip install -qU "langchain-chroma>=0.1.2"

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
import os
from google.colab import userdata

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="first",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
%pip install -qU langchain_community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
    "resume.pdf",
)

In [ ]:
docs = loader.load()
docs[0]

In [ ]:

vector_store.add_documents(documents=docs)

## making retriever

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)  # select top result

## making llm

In [ ]:
%pip install -qU langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## making prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

## chaining

In [ ]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

## querying

In [ ]:
response = rag_chain.invoke("which university does hamza  go to?")

print(response.content)

# face detection using a vector database

In [ ]:
!pip install -Uq facenet-pytorch

In [ ]:
!pip install -Uq pillow

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

import torchvision.transforms as transforms
from PIL import Image

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1

model = InceptionResnetV1(pretrained='vggface2').eval()
model

In [ ]:
# Preprocessing function to transform the image into a tensor
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

# Function to create image embeddings
def create_image_embedding(image_path):
    try:
        input_tensor = preprocess_image(image_path)
        with torch.no_grad():
            embeddings = model(input_tensor)# ebedding important line
        return embeddings.squeeze().numpy()
    except Exception as e:
        print("Error:", e)
        return None

In [ ]:
!mkdir images

In [ ]:
# prompt: create python function where we provide image url and imag_name then it save in images folder

import requests
import os

def save_image_from_url(image_url, image_name):
  """
  Downloads an image from a URL and saves it to the 'images' folder.

  Args:
    image_url: The URL of the image to download.
    image_name: The name of the file to save the image as.
  """
  try:
    if not os.path.exists("images"):
      os.makedirs("images")

    image_path = os.path.join("images", image_name)

    response = requests.get(image_url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(image_path, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"Image saved to: {image_path}")
  except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
  except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
save_image_from_url("https://avatars.githubusercontent.com/u/10209765?v=4", "q1.jpg")
save_image_from_url("https://media.licdn.com/dms/image/v2/D4D22AQFmuEiR8ttUmw/feedshare-shrink_800/feedshare-shrink_800/0/1711203894556?e=2147483647&v=beta&t=GEZGp_cdogNJCJIGidoEw_DjW2FXZcG4nUUlaNF1Zlc","z1.jpg")
save_image_from_url("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBBiqefc7Le97Rn0udVVBkur7RlU53FcQh1A&s",'z2.jpg')
save_image_from_url("https://i.ytimg.com/vi/7QD3GKvSyMk/hqdefault.jpg?sqp=-oaymwEmCOADEOgC8quKqQMa8AEB-AHOBYAC0AWKAgwIABABGGUgXChPMA8=&rs=AOn4CLB2EaZsLrClGHqUMUhApQ_sxAcF7Q","q2.jpg")

In [ ]:
# Example usage
image_path = "./images/q2.jpg"
q2 = create_image_embedding(image_path)

# 'embedding' now contains a dense vector representation of the image
print("Image Embedding Shape:", q2.shape)
print("Image Embedding:", q2)

In [ ]:
q1 = create_image_embedding("./images/q1.jpg")
q2 = create_image_embedding("./images/q2.jpg")
s1 = create_image_embedding("./images/s1.jpg")
s2 = create_image_embedding("./images/s2.jpg")
z1 = create_image_embedding("./images/z1.jpg")
z2 = create_image_embedding("./images/z2.jpg")

In [ ]:
!pip install -U milvus-lite

!pip install -U pymilvus


In [ ]:
from pymilvus import MilvusClient
client = MilvusClient("./milvus_demo.db")


In [ ]:
from pymilvus import MilvusClient
import numpy as np

client = MilvusClient("./milvus_demo.db")
client.create_collection(
    collection_name="images",
    dimension=512  # The vectors we will use in this demo has 384 dimensions
)

In [ ]:
data = [
    {"id": 1, "person_name": "Qasim", "vector": q1},
    {"id": 2, "person_name": "Qasim", "vector": q2},
    {"id": 3, "person_name": "Zia Khan", "vector": z1},
    {"id": 4, "person_name": "Zia Khan", "vector": z2}
]


In [ ]:
res = client.insert(
    collection_name="images",
    data=data
)

In [ ]:
res = client.search(
    collection_name="images",
    data=[q1],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)


In [ ]:
# t1 is qasim's same pic as q1
save_image_from_url("https://avatars.githubusercontent.com/u/10209765?v=4", "t1.jpg")
t1 = create_image_embedding('./images/t1.jpg')
t1


In [ ]:
res = client.search(
    collection_name="images",
    data=[t1],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)

In [ ]:
# t2 is bill gates pic
save_image_from_url("https://npr.brightspotcdn.com/dims3/default/strip/false/crop/4000x2667+0+0/resize/1100/quality/85/format/webp/?url=http%3A%2F%2Fnpr-brightspot.s3.amazonaws.com%2F82%2Ffb%2F62f7bcdd47329b5419411e9a7471%2Fbill-gates-portrait-at-npr.jpg", "t2.jpg")
t2 = create_image_embedding('./images/t2.jpg')
t2

In [ ]:
res = client.search(
    collection_name="images",
    data=[t2],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)

In [ ]:
# t3 is zia khan's new img
save_image_from_url("https://static.wixstatic.com/media/db6d86_c3ff0ba2fb6d45ad8d168621a2525cd0~mv2.png/v1/crop/x_0,y_18,w_2400,h_2340/fill/w_319,h_311,al_c,q_85,usm_0.66_1.00_0.01,enc_avif,quality_auto/Mr_%20Zia%20Khan.png", "t3.jpg")
t3 = create_image_embedding('./images/t3.jpg')
t3

In [ ]:
res = client.search(
    collection_name="images",
    data=[t3],
    limit=1,
    output_fields=["id", "person_name"],
)
print(res)

#VOICE DETECTION USING VECTOR DATABASE

In [ ]:
!pip install -Uq tensorflow_hub  pydub

In [ ]:
!pip install -Uq tensorflow-io

In [ ]:
# Step 1: Download the audio files
!mkdir audios
!wget -O audios/h1.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/hawking01.wav"
!wget -O audios/h2.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/hawking02.wav"
!wget -O audios/l1.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/king_injustice.wav"
!wget -O audios/l2.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/king_nonviolence.wav"
!wget -O audios/m1.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/manson_believe_me.wav"
!wget -O audios/m2.wav "https://www.wavsource.com/snds_2020-10-01_3728627494378403/people/famous/manson_devil.wav"



In [ ]:
# # Step 2: Install ffmpeg if not already installed
!apt-get install ffmpeg

# # Step 3: Convert audio files to 16-bit WAV
!ffmpeg -i audios/h1.wav -acodec pcm_s16le -ar 44100 audios/h1_16.wav
!ffmpeg -i audios/h2.wav -acodec pcm_s16le -ar 44100 audios/h2_16.wav
!ffmpeg -i audios/m1.wav -acodec pcm_s16le -ar 44100 audios/m1_16.wav
!ffmpeg -i audios/m2.wav -acodec pcm_s16le -ar 44100 audios/m2_16.wav
!ffmpeg -i audios/l1.wav -acodec pcm_s16le -ar 44100 audios/l1_16.wav
!ffmpeg -i audios/l2.wav -acodec pcm_s16le -ar 44100 audios/l2_16.wav


In [ ]:

!rm audios/h1.wav audios/h2.wav audios/m1.wav audios/m2.wav audios/l1.wav audios/l2.wav

In [ ]:

import tensorflow as tf
import tensorflow_hub as hub

# Load the YAMNET model
model = hub.load('https://tfhub.dev/google/yamnet/1')

# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audios/h1_16.wav'))
audio = tf.squeeze(audio, axis=-1)

# Generate embeddings
scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")

In [ ]:
# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audios/m1_16.wav'))
audio = tf.squeeze(audio, axis=-1)

scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")#27,18

In [ ]:
log_mel_spectrogram

#  we found shape dimension error
* you can solve with slicing

In [ ]:
import os
import numpy as np

voices = []
labels = []
for i in os.listdir("./audios/"):
    if '.wav' in i:
        name = i.split(".")[0]


        audio, sample_rate = tf.audio.decode_wav(tf.io.read_file(f'./audios/{i}'))
        audio = tf.squeeze(audio, axis=-1)

        scores, embeddings, log_mel_spectrogram = model(audio)

        voices.append(np.array(embeddings[:5,:]).ravel())
        labels.append(name)

        print(f"Audio embedding shape: {embeddings.shape} new shape {embeddings[:5,:].shape} type {type(np.array(embeddings[:18,:]))}")#27,18
        print(i)

In [ ]:
voices

In [ ]:
!pip install pymilvus

In [ ]:
#Imports a PyMilvus package:

from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

#Connect to the Milvus
connections.connect("default", host="localhost", port="19530")
# Define the collection name
collection_name = "audio"

# Delete old collection if it exists
if utility.has_collection(collection_name):
    Collection(collection_name).drop()


#Creates a collection:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="words", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="person_name", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=5120)
]
schema = CollectionSchema(fields, "Simple Demo for audio similar search")
audio = Collection("audio", schema)


In [ ]:
# Builds indexes on the entities:

index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

audio.create_index("embeddings", index)

In [ ]:
labels

In [ ]:
voices[0].shape

In [ ]:
#Insert data in collection
data = [
    [1,2,3,4,5,6],  # field pk
    labels,  # field words
    ["Auranzaib","Auranzaib","Hasnant","Qasim","Hasnant","Qasim"],
    voices,  # field embeddings
]

In [ ]:
audio.insert(data)
audio.flush()
audio.load()

In [ ]:
search_params = {"metric_type": "L2"}

In [ ]:
results = audio.search(
	data=[voices[0]],
	anns_field="embeddings",
	param=search_params,
	limit=4,
	expr=None,
	# set the names of the fields you want to retrieve from the search result.
	output_fields=['words','person_name'],
	consistency_level="Strong"
)

In [ ]:
for i in range(0,len(results[0])):
    name = results[0][i].entity.get('words')
    pname = results[0][i].entity.get('person_name')
    print(pname)
#     try:
#         display(Image.open('./images/'+name+'.jpg'))
#     except:
#         display(Image.open('./images/'+name+'.png'))

In [ ]:
labels

# Langchain Function Calling

In [ ]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY_1')

In [ ]:
%pip install -qU langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

print(messages)




In [ ]:
print(llm_with_tools.invoke(messages).content)

# Langchain tool calling

In [ ]:
!pip install --upgrade stackapi

In [ ]:
from langchain_community.utilities import StackExchangeAPIWrapper

@tool
def stackexchange(query: str) -> str:
  """Executes a search query on StackExchange and returns relevant answers or posts.
  - `query`: The user's search or troubleshooting question.
  - Returns: Top results from StackExchange."""
  return StackExchangeAPIWrapper().run(query)

In [ ]:
tools = [stackexchange]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage

query = "zsh: command not found: python"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"stackexchange": stackexchange}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

print(messages)




In [ ]:
print(llm_with_tools.invoke(messages).content)